# 3D Voxel Visualization from PDB

## 1. Install Dependencies (Optional)Make sure you have installed the following Python packages:```bashpip install biopython numpy pandas matplotlib scikit-image torch torchvision tqdm scipy```

## 2. Import Modules and Define Constants

In [ ]:
import osimport pickleimport matplotlib.pyplot as pltimport numpy as npimport pandas as pdfrom Bio.PDB import is_aa, PDBParserfrom Bio.PDB.Polypeptide import protein_letters_3to1from mpl_toolkits.mplot3d.art3d import Poly3DCollectionfrom skimage import measurefrom torch.utils.data import Datasetfrom torchvision import transformsfrom tqdm import tqdmfrom scipy.ndimage import map_coordinatesfrom matplotlib.colors import Normalize# --- The following constants should be defined according to your project ---# AMINO_ACID_WATER: dict, 1-letter amino acid -> water affinity# AMINO_ACID_CHARGE: dict, 1-letter amino acid -> charge value# AMAs: dict, 1-letter amino acid -> category index# ATOMS: dict, atom name (e.g. 'CA') -> weight# ATOMS_R: dict, atom name -> radius (int)# clamp: coordinate clipping function, e.g.:def clamp(x, min_val, max_val):    return max(min(x, max_val), min_val)

## 3. Define the `pdb_parser` Function

In [ ]:
def pdb_parser(parser, idx, pdb_path):    """    Parse the PDB file, map structure to a (3×64×64×64) voxel grid,    and return the amino acid sequence string.    """    # Initialize voxel grid: 3 channels (weight, water, charge)    voxel = np.zeros((3, 64, 64, 64), dtype=np.int8)    structure = parser.get_structure(idx, pdb_path)    # Get chain ID    chain_id = None    for model in structure:        for chain in model:            chain_id = chain.id    chain = structure[0][chain_id]    seq_str = ''    # Iterate over residues    for res in chain:        if is_aa(res.get_resname(), standard=True):            amino = protein_letters_3to1[res.get_resname()]            seq_str += amino            ATOM_WATER = AMINO_ACID_WATER[amino]            ATOM_CHARGE = AMINO_ACID_CHARGE[amino]            # ATOM_CATEGORY = AMAs[amino] * 20  # optional extra channel            for atom in res:                if atom.id not in ATOMS:                    continue                x, y, z = atom.get_coord()                # Clip coordinates                if abs(x) > 32: x = clamp(x, -31, 31)                if abs(y) > 32: y = clamp(y, -31, 31)                if abs(z) > 32: z = clamp(z, -31, 31)                xi, yi, zi = int(x) + 32, int(y) + 32, int(z) + 32                ATOM_WEIGHT = ATOMS[atom.id]                ATOM_R = ATOMS_R[atom.id]                # Fill voxels based on radius                if ATOM_R == 1:                    rng = slice(xi-1, xi)                elif ATOM_R <= 1.5:                    rng = slice(xi-1, xi+1)                else:                    r = int(ATOM_R)                    rng = slice(xi-r, xi+r)                voxel[0][rng, rng, rng] = ATOM_WEIGHT                voxel[1][rng, rng, rng] = ATOM_WATER                voxel[2][rng, rng, rng] = ATOM_CHARGE    return voxel, seq_str

## 4. Define the `visualize_voxel_3d_rgb` Function

In [ ]:
def visualize_voxel_3d_rgb(pdb_path: str,                           output_jpg: str,                           iso_channel: int = 0,                           threshold: float = 0.5,                           figsize: tuple = (8, 8),                           dpi: int = 300):    """    Parse PDB file into voxels, extract isosurface, and save as JPEG    colored by RGB channels.    """    parser = PDBParser(QUIET=True)    voxel, _ = pdb_parser(parser, 'struct', pdb_path)    volumes = [voxel[i].astype(np.float32) for i in range(3)]    # 2) Extract isosurface    verts, faces, normals, vals = measure.marching_cubes(        volumes[iso_channel],        level=threshold,        spacing=(1.0, 1.0, 1.0)    )    # 3) Sample all three channels at vertices    coords = np.vstack([verts[:,2], verts[:,1], verts[:,0]])    sampled = np.zeros((verts.shape[0], 3), dtype=np.float32)    for ch in range(3):        sampled[:, ch] = map_coordinates(            volumes[ch],            coords,            order=1,            mode='nearest'        )    # 4) Average values of its three vertices per face    face_vals = sampled[faces].mean(axis=1)    # 5) Normalize to [0,1]    norms = [Normalize(vmin=v.min(), vmax=v.max()) for v in volumes]    face_colors = np.zeros((faces.shape[0], 4), dtype=np.float32)    for ch in range(3): face_colors[:, ch] = norms[ch](face_vals[:, ch])    face_colors[:, 3] = 1.0    # 6) Plot    fig = plt.figure(figsize=figsize)    ax = fig.add_subplot(111, projection='3d')    mesh = Poly3DCollection(        verts[faces],        facecolors=face_colors,        linewidth=0.05,        edgecolor='gray'    )    ax.add_collection3d(mesh)    lim = volumes[0].shape[0]    ax.set_xlim(0, lim)    ax.set_ylim(0, lim)    ax.set_zlim(0, lim)    ax.set_xlabel('X')    ax.set_ylabel('Y')    ax.set_zlabel('Z')    ax.set_title(f'RGB Isosurface (ch={iso_channel}, thr={threshold})')    ax.text2D(        0.02, 0.95,        'R=weight (ch0), G=water (ch1), B=charge (ch2)',        transform=ax.transAxes    )    plt.tight_layout()    plt.savefig(output_jpg, dpi=dpi)    plt.close(fig)

## 5. Example Usage

In [ ]:
# Replace 'example.pdb' with your PDB file pathpdb_file = 'example.pdb'output_file = 'example.jpg'# Adjust iso_channel (0,1,2) and threshold for different effectsvisualize_voxel_3d_rgb(pdb_file, output_file, iso_channel=0, threshold=0.3)print(f'Visualization saved to: {output_file}')